# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [55]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

### 데이터 셋 읽어오기

In [56]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [57]:
# 중복된 행을 찾음 (모든 열 값이 같은 행)
def duplicated(df_train):
    duplicate_rows_df = df_train[df_train.duplicated(keep=False)]

    # 중복된 행의 개수
    duplicate_row_count = duplicate_rows_df.shape[0]

    # 중복된 행이 있는 경우, 그 행들의 종류와 개수를 출력
    if duplicate_row_count > 0:
        # 중복된 행들의 종류와 각각의 개수를 세기
        duplicate_row_types = duplicate_rows_df.drop_duplicates()
        duplicate_row_types_count = duplicate_row_types.shape[0]

        print(f"Number of duplicate row pairs: {duplicate_row_count // 2}")
        print(f"Number of unique duplicate row types: {duplicate_row_types_count}")
        print("Unique duplicate row types:")
        print(duplicate_row_types)
    else:
        print("No duplicate rows found.")
duplicated(df_train)
df_train=df_train.drop_duplicates(keep='first', ignore_index = True)
duplicated(df_train)

Number of duplicate row pairs: 2962
Number of unique duplicate row types: 2405
Unique duplicate row types:
       bant_submit           customer_country business_unit  \
319           1.00  /East London/South Africa            AS   
321           1.00  /East London/South Africa            AS   
343           1.00        /Medellin /Colombia            AS   
1252          1.00        /Brisbane/Australia            ID   
1313          1.00                    //Ghana            ID   
...            ...                        ...           ...   
59133         0.75                   //Mexico            ID   
59261         1.00    /rio de janeiro /Brazil            AS   
59274         1.00              /Temuco/Chile            AS   
59289         0.75       /Dolnośląskie/Poland            AS   
59293         1.00            /Sląskie/Poland            AS   

       com_reg_ver_win_rate  customer_idx           customer_type  enterprise  \
319                0.040816         30958            En

In [58]:
category_columns = [
    "customer_country",
    "business_unit",
    "customer_idx",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "business_subarea",
    "business_area",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "expected_timeline",
    "response_corporate",
    "lead_owner"
]


In [59]:
#process country
df_train["customer_country"] = df_train["customer_country"].str.split('/').str[-1].str.strip()
rows_to_remove = df_train["customer_country"].str.contains('[0-9!@#$%^&*(),.?":{}|<>]', na=False, regex=True)
df_train.loc[rows_to_remove, "customer_country"] = ''

df_test["customer_country"] = df_test["customer_country"].str.split('/').str[-1].str.strip()
rows_to_remove = df_test["customer_country"].str.contains('[0-9!@#$%^&*(),.?":{}|<>]', na=False, regex=True)
df_test.loc[rows_to_remove, "customer_country"] = ''

#process customer job not sure to use this
df_train["customer_job"] = df_train["customer_job"].str.split('/').str[0].str.strip()
df_test["customer_job"] = df_test["customer_job"].str.split('/').str[0].str.strip()

#process category
df_train["product_category"] = df_train["product_category"].str.replace('[^a-zA-Z0-9\s]', '', regex=True)
df_test["product_category"] = df_test["product_category"].str.replace('[^a-zA-Z0-9\s]', '', regex=True)

mask = df_train['product_modelname'].isin(df_train['product_category'].unique())
df_train.loc[mask, 'product_modelname'] = df_train['product_category']
mask = df_test['product_modelname'].isin(df_test['product_category'].unique())
df_test.loc[mask, 'product_modelname'] = df_test['product_category']

#process modelname
df_train["product_modelname"] = df_train["product_modelname"].str.replace(r'\([^)]*\)', '', regex=True).str.strip()
df_test["product_modelname"] = df_test["product_modelname"].str.replace(r'\([^)]*\)', '', regex=True).str.strip()
df_train["product_modelname"] = df_train["product_modelname"].str.replace(r'-[^ ]*', '', regex=True).str.strip()
df_test["product_modelname"] = df_test["product_modelname"].str.replace(r'-[^ ]*', '', regex=True).str.strip()


#process expected timeline
df_train["expected_timeline"] = df_train["expected_timeline"].str.replace('_', ' ').str.rstrip('.')
df_test["expected_timeline"] = df_test["expected_timeline"].str.replace('_', ' ').str.rstrip('.')
df_train["expected_timeline"] = df_train["expected_timeline"].str.split().str[:3].str.join(' ')
df_test["expected_timeline"] = df_test["expected_timeline"].str.split().str[:3].str.join(' ')
df_train["expected_timeline"] = df_train["expected_timeline"].str.replace(r'[.,/].*', '', regex=True)
df_test["expected_timeline"] = df_test["expected_timeline"].str.replace(r'[.,/].*', '', regex=True)


#### customer country NLP로 전처리후 모델에 넣으면 학습 어떻게 되는지 확인해야됨
#### 카테고리 데이터 숫자로 변환후 원핫 인코딩 진행 및 수치형 데이터 노말라이즈 진행(train기준 maxmin으로 진행)

## 2. 데이터 전처리

### 레이블 인코딩

In [60]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [61]:
# # 레이블 인코딩할 칼럼들
# label_columns = [
#     "customer_country",
#     "business_subarea",
#     "business_area",
#     "business_unit",
#     "customer_type",
#     "enterprise",
#     "customer_job",
#     "inquiry_type",
#     "product_category",
#     "product_subcategory",
#     "product_modelname",
#     "customer_country.1",
#     "customer_position",
#     "response_corporate",
#     "expected_timeline",
# ]
# category_columns = [
#     "customer_country",
#     "business_unit",
#     "customer_idx",
#     "customer_type",
#     "enterprise",
#     "customer_job",
#     "inquiry_type",
#     "business_subarea",
#     "business_area",
#     "product_category",
#     "product_subcategory",
#     "product_modelname",
#     "customer_country.1",
#     "customer_position",
#     "expected_timeline",
#     "response_corporate",
#     "lead_owner"
# ]

# label_mappings = {}

# for col in category_columns:
#     if col in df_train.columns:
#         # df_train에서의 빈도수 계산 및 5회 미만을 NaN으로 매핑
#         value_counts = df_train[col].dropna().value_counts()
#         values_to_zero = value_counts[value_counts < 5].index
#         df_train.loc[df_train[col].isin(values_to_zero), col] = np.nan

#         # 빈도수에 따라 내림차순으로 레이블 할당
#         value_counts = df_train[col].dropna().value_counts().sort_values(ascending=False)
#         new_labels = {np.nan: 0}
        
#         # 가장 빈도수가 높은 카테고리가 가장 큰 값을 가지도록 레이블 카운트 설정
#         new_label_count = len(value_counts)
#         for value in value_counts.index:
#             new_labels[value] = new_label_count
#             new_label_count -= 1  # 레이블 값을 감소시키며 할당

#         # 레이블 매핑 저장
#         label_mappings[col] = new_labels

#         # df_train에 레이블 매핑 적용
#         df_train[col] = df_train[col].map(new_labels)

# # df_test에 대한 새로운 카테고리의 빈도수 계산 및 레이블 매핑 적용
# for col, mapping in label_mappings.items():
#     if col in df_test.columns:
#         # df_test에서 각 카테고리의 빈도수 계산
#         test_value_counts = df_test[col].value_counts()

#         # df_train에서 사용된 최대 레이블 번호 확인
#         max_label = max(mapping.values())

#         # 새로운 카테고리에 대해 레이블 매핑 적용
#         new_mapping = mapping.copy()  # 기존 매핑 복사
#         for value, count in test_value_counts.items():
#             # 새로운 카테고리이고 5회 이상 나타나는 경우 새로운 레이블 할당
#             if value not in new_mapping and count >= 5:
#                 print(value)
#                 max_label += 1
#                 new_mapping[value] = max_label

#         # df_test에 새로운 레이블 매핑 적용
#         df_test[col] = df_test[col].apply(lambda x: new_mapping.get(x, 0))  # 매핑에 없는 경우 0으로 처리


In [62]:

label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
     df_all[col] = label_encoding(df_all[col])
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]
#df_train.dropna(axis=1, inplace=True)
#df_test.dropna(axis=1, inplace=True)# 레이블 인코딩할 칼럼들

#### 5회 미만 제거 및 범주형 데이터 빈도수에 맞추어 labeling

다시 학습 데이터와 제출 데이터를 분리합니다.

In [63]:
# for col in label_columns:  
#     df_train[col] = df_all.iloc[: len(df_train)][col]
#     df_test[col] = df_all.iloc[len(df_train) :][col]
# #df_train.dropna(axis=1, inplace=True)
# #df_test.dropna(axis=1, inplace=True)

### VIF 다중공선성 해결

In [64]:
from sklearn.linear_model import LinearRegression as LR
df_train.drop(['customer_country.1'],axis=1,inplace=True)
df_test.drop(['customer_country.1'],axis=1,inplace=True)


VIF_dict = dict()
df_train.fillna(0,inplace=True) # 결측치 -1로 채워봄
df_test.fillna(0,inplace=True)
for col in df_train.columns:
    model = LR().fit(df_train.drop([col], axis=1), df_train[col])
    r2 = model.score(df_train.drop([col], axis=1), df_train[col])
    VIF = 1 / max((1 - r2), 1e-9)
    VIF_dict[col] = VIF

# VIF 값이 5 이상인 열을 출력
high_VIF_columns = [col for col, vif in VIF_dict.items() if vif >= 5]
print("Columns with VIF >= 5:", high_VIF_columns)


Columns with VIF >= 5: ['id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver']


In [65]:
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
# XGBoost 분류기를 훈련합니다.
model = XGBClassifier()
model.fit(df_train.drop('is_converted', axis=1), df_train['is_converted'])

# Feature importance를 가져옵니다.
importances = model.feature_importances_

# Feature importance를 DataFrame으로 변환합니다.
feature_importance_df = pd.DataFrame({
    'Feature': df_train.drop('is_converted', axis=1).columns,
    'Importance': importances
})




### 2-2. 학습, 검증 데이터 분리

#### tomek links

In [66]:
'''
# 수치형 컬럼의 NaN 값을 평균으로 대체
# 수치형 컬럼을 식별합니다.
numeric_columns = list(set(df_train.columns) - set(label_columns))

for col in numeric_columns:
    if df_train[col].isnull().any():
        df_train[col].fillna(df_train[col].mean(), inplace=True)
categorical_features_indices=[]
# 범주형 컬럼의 NaN 값을 가장 빈번한 값으로 대체
for col in label_columns:
    if df_train[col].isnull().any():
        df_train[col].fillna(df_train[col].mode()[0], inplace=True)
    categorical_features_indices=df_train.columns.get_loc(col)
    
'''

'\n# 수치형 컬럼의 NaN 값을 평균으로 대체\n# 수치형 컬럼을 식별합니다.\nnumeric_columns = list(set(df_train.columns) - set(label_columns))\n\nfor col in numeric_columns:\n    if df_train[col].isnull().any():\n        df_train[col].fillna(df_train[col].mean(), inplace=True)\ncategorical_features_indices=[]\n# 범주형 컬럼의 NaN 값을 가장 빈번한 값으로 대체\nfor col in label_columns:\n    if df_train[col].isnull().any():\n        df_train[col].fillna(df_train[col].mode()[0], inplace=True)\n    categorical_features_indices=df_train.columns.get_loc(col)\n    \n'

In [67]:

# TRUE와 FALSE 개수 세기
true_count = df_train['is_converted'].sum()
false_count = len(df_train) - true_count

# 두 개수 중 작은 값으로 데이터를 분할
min_count = min(true_count, false_count)

# TRUE와 FALSE 개수를 맞추어 데이터를 분할
true_data = df_train[df_train['is_converted'] == True].sample(n=min_count, random_state=400)
false_data = df_train[df_train['is_converted'] == False].sample(n=min_count, random_state=400)

# 데이터를 결합
df_balanced = pd.concat([true_data, false_data])

# val set을 먼저 구성
val_size = int(len(df_balanced) * 0.2)  # 전체 데이터의 20%를 val set으로 사용
val_set = df_balanced.sample(n=val_size, random_state=400)

# val_set을 x_val과 y_val로 분리
x_val = val_set.drop("is_converted", axis=1)
y_val = val_set["is_converted"]

# train set 구성 (비율을 맞추기 전의 데이터 사용)
train_set = df_train.drop(val_set.index)

# train set과 val set 구성 확인
print("Train set:")
print(train_set['is_converted'].value_counts())
print("Validation set:")
print(val_set['is_converted'].value_counts())


Train set:
False    50245
True      3687
Name: is_converted, dtype: int64
Validation set:
True     933
False    915
Name: is_converted, dtype: int64


In [68]:

from imblearn.under_sampling import TomekLinks, RandomUnderSampler
import pandas as pd


# Tomek Link를 적용하여 데이터 보정
tl = TomekLinks()
x_train_resampled, y_train_resampled = tl.fit_resample(train_set.drop("is_converted", axis=1), train_set["is_converted"])

# 클래스 비율 맞추기
rus = RandomUnderSampler(sampling_strategy={0: int(y_train_resampled.sum() * 1), 1: y_train_resampled.sum()}, random_state=42)
x_train, y_train = rus.fit_resample(x_train_resampled, y_train_resampled)

# 결과 확인
print("Train set after Tomek Links and balancing classes:")
print(pd.Series(y_train).value_counts())


Train set after Tomek Links and balancing classes:
False    3687
True     3687
Name: is_converted, dtype: int64


In [69]:
# x_train, x_val, y_train, y_val = train_test_split(
#     df_train.drop("is_converted", axis=1),
#     df_train["is_converted"],
#     test_size=0.2,
#     shuffle=True,
#     random_state=400,
# )

## 3. 모델 학습

### 모델 정의 

In [70]:

#!pip install shap
#model = DecisionTreeClassifier()
#model = DecisionTreeClassifier(max_depth=12, min_samples_split=3, min_samples_leaf=5)
#model = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, min_samples_leaf=2)
true_count = sum(df_train['is_converted'] == True)
false_count = sum(df_train['is_converted'] == False)
scale_pos_weight = false_count / true_count
#scale_pos_weight=scale_pos_weight*0.5
# XGBoost 모델 초기화
model = XGBClassifier()


model_param_grid = {
    'n_estimators': [100, 150,200],
    'learning_rate': [0.1, 0.15, 0.2, 0.3],
    'max_depth': [6, 8, 10, 12]
}

model_grid=GridSearchCV(model, param_grid = model_param_grid, scoring="accuracy", n_jobs=-1, verbose = 1)
model_grid.fit(x_train, y_train)
model = model_grid.best_estimator_
model.fit(x_train, y_train) 

# import shap


# # SHAP 값을 계산하기 위한 Explainer 생성
# explainer = shap.Explainer(model, x_train)
# x_test = df_test.drop(["is_converted", "id"], axis=1)
# # 테스트 세트에 대한 SHAP 값 계산
# shap_values = explainer(x_test)

# # SHAP 요약 플롯
# shap.summary_plot(shap_values, x_test)

# # 특정 인스턴스에 대한 SHAP 값 시각화 (예: 테스트 세트의 첫 번째 인스턴스)
# shap.waterfall_plot(explainer.expected_value, shap_values[0].values, feature_names=x_test.columns.tolist())


Fitting 5 folds for each of 48 candidates, totalling 240 fits


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.15, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

### 모델 학습

In [71]:
model.fit(x_train.fillna(0), y_train) # -1 로 바꿔뒀음

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.15, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

### 모델 성능 보기

In [72]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, pred)
    TP = confusion[1][1]
    FP = confusion[0][1]
    TN = confusion[0][0]
    FN = confusion[1][0]
    print(f"True Positive(TP): {TP}, False Positive(FP): {FP}, True Negative(TN): {TN}, False Negative(FN): {FN}")
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [73]:
#pred = model.predict(x_val_selected_features.fillna(0))
pred = model.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)
print(type(model))
#print("트리의 최대 깊이:", model.tree_.max_depth)


True Positive(TP): 874, False Positive(FP): 68, True Negative(TN): 847, False Negative(FN): 59
오차행렬:
 [[847  68]
 [ 59 874]]

정확도: 0.9313
정밀도: 0.9278
재현율: 0.9368
F1: 0.9323
<class 'xgboost.sklearn.XGBClassifier'>


## 4. 제출하기

### 테스트 데이터 예측

In [74]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [75]:
test_pred = model.predict(x_test.fillna(0))
sum(test_pred) # True로 예측된 개수

1980

### 제출 파일 작성

In [76]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

#in vscode gitignore issue

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**